In [28]:
import pandas as pd
import numpy as np

In [29]:
#Reading Data
isi_auftrag = pd.read_csv("../data/ISI_Daten/ISI_Auftragsdetails_s822.csv", sep="|")
isi_aufenthalt = pd.read_csv("../data/ISI_Daten/ISI_FzAufenthalte.csv", sep="|")
isi_stamm = pd.read_csv("../data/ISI_Daten/ISI_Stammdaten.csv", sep="|")

# Auftragsdetails

In [30]:
#Duplikate entfernen
isi_auftrag = isi_auftrag.drop_duplicates()

#Spaltennamen anpassen
isi_auftrag = isi_auftrag.rename(columns={'FZ_NR': 'FZ-ID'})

#Unnötige Spalten löschen
isi_auftrag = isi_auftrag.drop(['WERK'], axis=1)

# Aufenthalt

In [31]:
#Duplikate entfernen
isi_aufenthalt = isi_aufenthalt.drop_duplicates()

#WHO-Nummer umrechnen
isi_aufenthalt['WHO_NUMMER'] = isi_aufenthalt['WHO_NUMMER'] - 1000000000

#Unnötige Spalten entfernen
isi_aufenthalt = isi_aufenthalt.drop(['FZ_TYP', 'HZG', 'DAT_EINBUCH', 'PLAN_ÜBERGABE_1', 'PLAN_ÜBERGABE_2'], axis=1)

#Timestamp "DAT_AUSBUCH" zu Integer wechseln
isi_aufenthalt["DAT_AUSBUCH"] = isi_aufenthalt.apply(
    lambda row: 0 if np.isnan(row['DAT_AUSBUCH']) else row['DAT_AUSBUCH'],
    axis=1)

isi_aufenthalt["DAT_AUSBUCH"] = isi_aufenthalt["DAT_AUSBUCH"].astype(int)

# Stammdaten

In [32]:
#Duplikate entfernen
isi_stamm = isi_stamm.drop_duplicates()

#Busmaße in Meter umwandeln
isi_stamm['FZ_LAENGE'] = isi_stamm['FZ_LAENGE'] / 1000
isi_stamm['FZ_BREITE'] = isi_stamm['FZ_BREITE'] / 1000
isi_stamm['FZ_HOEHE'] = isi_stamm['FZ_HOEHE'] / 1000

#Unnötige Spalten löschen
isi_stamm = isi_stamm.drop(['WERKSTATT_NAME', 'CHASSIS_BAUART', 'D2_TYP', 'KRAFTSTOFF_ART', 'NENNLEISTUNG', 'MARKE', 'ZULASSUNGSDATUM', 'REISEGESCHWINDGK'], axis=1)

#Plätze kombinieren und alte Spalten löschen
isi_stamm['PLAETZE'] = isi_stamm['SITZPLAETZE'] + isi_stamm['STEHPLAETZE']
isi_stamm = isi_stamm.drop(['SITZPLAETZE','STEHPLAETZE' ], axis=1)

#Inaktive Fahrzeuge entfernen
isi_stamm = isi_stamm[isi_stamm['AKTIV'] != 0]

#Klima kategorisieren
isi_stamm['KLIMAANLAGE'] = isi_stamm['KLIMAANLAGE'].astype('category').cat.codes
isi_stamm['KLIMAANLAGE'] = isi_stamm['KLIMAANLAGE'] + 1

# Clean Aufträge

In [33]:
isi_auftraege = pd.read_csv("../data/ISI_Daten/ISI_Auftraege.csv", sep="|")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
#Duplikate entfernen
isi_auftraege = isi_auftraege.drop_duplicates()

#Unnötige Spalten löschen
isi_auftraege = isi_auftraege.drop(['BAHNSTELLE', 'ARBEITSPLATZ', 'ILA', 'AUSGANG'], axis=1)

#WHO-Nummer umrechnen
isi_auftraege['WHO_NUMMER'] = isi_auftraege['WHO_NUMMER'] - 1000000000


#Anpassungen
isi_auftraege = isi_auftraege.rename(columns={'FZ_NR': 'FZ-ID'})

In [35]:
isi_auftraege.head(1)

,FZ-ID,WERK,AUFTRAG,AUFTRAGSTEXT,WHO_NUMMER,AUFTRAGSART,SYSTEMSTATUS,ECKSTARTTERMIN,ECKENDTERMIN,BEZUGSDATUM,UPDATE_DATE,UEBERGEORDN_AUFTRAG
0,1d707286-e40a-4d8d-8c5d-47cff0169954,NaN,1032929649,Inst.schw.Unfallsch./G3.9 (Gw),158,610M,ABGS RÜCK ABRV MABS TDRU VOKL WABE,43102,43102,43144,43510.536806,NaN


# Angleichung der jewweils vorhandenen FZ-IDS

In [36]:
isi_stamm = isi_stamm[isi_stamm['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [37]:
isi_auftraege = isi_auftraege[isi_auftraege['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [38]:
isi_aufenthalt = isi_aufenthalt[isi_aufenthalt['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [39]:
isi_aufrag = isi_aufenthalt[isi_aufenthalt['FZ-ID'].isin(isi_aufenthalt['FZ-ID'])]

In [40]:
isi_stamm = isi_stamm[isi_stamm['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [41]:
isi_auftraege = isi_auftraege[isi_auftraege['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [42]:
isi_aufenthalt = isi_aufenthalt[isi_aufenthalt['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

# IDs Filtern (?)

In [43]:
ids_auftrag = isi_auftrag['FZ-ID'].unique()
ids_aufenthalt = isi_aufenthalt['FZ-ID'].unique()
ids_stamm = isi_stamm['FZ-ID'].unique()


common_ids = list(set(ids_auftrag) & set(ids_aufenthalt) & set(ids_stamm))
common_ids =list(map(str, common_ids))


isi_auftrag = isi_auftrag[~isi_auftrag['FZ-ID'].isin(common_ids)]
isi_aufenthalt = isi_aufenthalt[~isi_aufenthalt['FZ-ID'].isin(common_ids)]
isi_stamm = isi_stamm[~isi_stamm['FZ-ID'].isin(common_ids)]


In [44]:
pd.DataFrame(common_ids).to_csv("../Cleaned_Data/ids.csv", index = "FZ-ID")

In [45]:
ids = pd.read_csv("../Cleaned_Data/ids.csv", sep=",", index_col=0)

In [46]:
ids = ids.rename(columns={'o': 'FZ-ID'})

In [47]:
# Neue Daten speichern

In [48]:
isi_stamm.to_csv("../Cleaned_Data/isi_stammdaten.csv", index = False)
isi_aufenthalt.to_csv("../Cleaned_Data/isi_aufenthalt.csv", index = False)
isi_auftrag.to_csv("../Cleaned_Data/isi_auftrag.csv", index = False)
isi_auftraege.to_csv("../Cleaned_Data/isi_auftraege.csv", index = False)

In [49]:
ids.to_csv("../Cleaned_Data/ids.csv", index = False)